In [5]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import IPython.display as ipd
import plotly.express as px
import pandas as pd
import numpy as np
import warnings
import glob
import IPython
import os
from tqdm import tqdm

#必要なライブラリの読み込み
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
import os
from glob import glob
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
import pandas as pd
import cv2

warnings.simplefilter('ignore')

plt.style.use("ggplot")

In [2]:
#画像のサイズ指定
ScaleTo = 70
seed = 7
n_categories = 3

#トレーニングデータの読み込み
data_dir ="input/train"
path = "input/train/*/*.jpg"
files = glob(path)

trainImg = []
trainLabel = []
j = 1
num = len(files)

#画像データをリストに格納
for img in files:
    print(str(j) + "/" + str(num) , end="\r")
    trainImg.append(cv2.resize(cv2.imread(img) ,(ScaleTo,ScaleTo)))
    #trainLabel.append(img.split("/")[-2])
    j += 1


#ラベルをリストに格納
for dir in os.listdir(data_dir):
    if dir == ".DS_Store":
        continue
    dir1 = data_dir + "/" + dir
    label = dir

    for file in os.listdir(dir1):
        if file != "Thumbs.db":

            trainLabel.append(label)

In [4]:
# kerasに渡すためにnumpy配列に変換。
image_list = np.asarray(trainImg)
label_list = pd.DataFrame(trainLabel)


clearTrainImg = trainImg

clearTrainImg = np.asarray(clearTrainImg)
clearTrainImg.shape

# クラスの形式を変換
le = LabelEncoder()
le = le.fit(label_list)
label_list = le.transform(label_list)
label_list
label_list = np_utils.to_categorical(label_list)
label_list

X_train, X_test, y_train, y_test = train_test_split(clearTrainImg, label_list, test_size=0.33, random_state=0)



from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=180,  # randomly rotate images in the range
        zoom_range = 0.1, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically
        horizontal_flip=True,  # randomly flip images horizontally
        vertical_flip=True  # randomly flip images vertically
    )
datagen.fit(X_train)











#モデル作成・学習
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import tensorflow as tf
import keras
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D,Input
from keras.layers import Dense, Dropout, Flatten, Activation,GlobalAveragePooling2D,Input
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam
from keras import optimizers


# ResNet50のロード。FC層は不要なので include_top=False
input_tensor = Input(shape=(ScaleTo, ScaleTo, 3))
resnet50 = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)

# FC層の作成
top_model = Sequential()
top_model.add(Flatten(input_shape=resnet50.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
#top_model.add(Dense(nb_classes, activation='softmax'))
top_model.add(Dense(n_categories, activation='softmax'))

# ResNet50とFC層を結合してモデルを作成
resnet50_model = Model(input=resnet50.input, output=top_model(resnet50.output))

#ResNet50の一部の重みを固定
for layer in resnet50_model.layers[:100]:
    layer.trainable = False

# 多クラス分類を指定
resnet50_model.compile(loss='categorical_crossentropy',
          optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
          metrics=['accuracy'])
resnet50_model.summary()


#学習の実行
#hist = resnet50_model.fit_generator(datagen.flow(X_train, y_train, batch_size=75),
#                        epochs=35, validation_data=(X_test, y_test),
#                        steps_per_epoch=X_train.shape[0])

resnet50_model.fit(datagen.flow(X_train, y_train, batch_size=75),epochs=35)


#モデルの評価
print(resnet50_model.evaluate(X_train, y_train)) #トレーニングの精度
print(resnet50_model.evaluate(X_test, y_test))  #テスト精度


#パラメータの保存
model.save_weights('../content/weights.h5')

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 70, 70, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 76, 76, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 35, 35, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 35, 35, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv1_1/convolution (defined at C:\Users\organ\Anaconda3\envs\keras_env\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_41213]

Function call stack:
keras_scratch_graph
